<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/v3-lstm-4hours-fluencybank-duplicate-tie-outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy pandas tensorflow

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report

# Load the data
data = pd.read_csv('MFCC_with_Output.csv')

data['MFCC'] = data['MFCC'].apply(lambda x: np.array([float(num) for num in x.strip('[]').split(',')]))

# Preparing the dataset
X = np.array(data['MFCC'].tolist())
y = data['Output'].values

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the labels
y_encoded = to_categorical(y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Building the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(y_encoded.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
# Train the model
model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/300
137/137 [==============================] - 12s 29ms/step - loss: 2.1470 - accuracy: 0.3511 - val_loss: 1.8568 - val_accuracy: 0.3942
Epoch 2/300
137/137 [==============================] - 2s 14ms/step - loss: 1.8073 - accuracy: 0.4008 - val_loss: 1.7207 - val_accuracy: 0.3942
Epoch 3/300
137/137 [==============================] - 2s 13ms/step - loss: 1.7661 - accuracy: 0.4026 - val_loss: 1.6998 - val_accuracy: 0.3942
Epoch 4/300
137/137 [==============================] - 2s 16ms/step - loss: 1.7472 - accuracy: 0.4013 - val_loss: 1.6833 - val_accuracy: 0.3942
Epoch 5/300
137/137 [==============================] - 3s 19ms/step - loss: 1.7295 - accuracy: 0.4020 - val_loss: 1.6725 - val_accuracy: 0.3942
Epoch 6/300
137/137 [==============================] - 2s 17ms/step - loss: 1.7171 - accuracy: 0.4086 - val_loss: 1.6640 - val_accuracy: 0.3914
Epoch 7/300
137/137 [==============================] - 2s 15ms/step - loss: 1.7122 - accuracy: 0.4068 - val_loss: 1.6583 - val_accuracy

In [4]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {scores[1]*100}%')

Accuracy: 42.79197156429291%


In [5]:
# Predict classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate F1 score for each class
f1_scores = f1_score(y_true_classes, y_pred_classes, average=None)  # Returns an array of F1 scores for each class

# Print F1 scores for each class
for idx, f1 in enumerate(f1_scores):
    print(f'F1 Score for Class {idx}: {f1:.4f}')

# Optionally, print a full classification report
print(classification_report(y_true_classes, y_pred_classes))

35/35 [==============================] - 1s 3ms/step
F1 Score for Class 0: 0.0000
F1 Score for Class 1: 0.1481
F1 Score for Class 2: 0.0952
F1 Score for Class 3: 0.2780
F1 Score for Class 4: 0.0396
F1 Score for Class 5: 0.1250
F1 Score for Class 6: 0.2954
F1 Score for Class 7: 0.6026
F1 Score for Class 8: 0.0000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       0.38      0.09      0.15        87
           3       0.16      0.07      0.10       102
           4       0.39      0.22      0.28       143
           5       0.40      0.02      0.04        96
           6       0.16      0.10      0.12        29
           7       0.37      0.25      0.30       195
           8       0.46      0.86      0.60       432
           9       0.00      0.00      0.00        11

    accuracy                           0.43      1096
   macro avg       0.26      0.18      0.18      1096
weighted avg       0.38      0.43   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
